In [10]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.activations import relu, tanh, sigmoid, softmax
from sklearn.model_selection import train_test_split

In [2]:
crop_data = pd.read_csv('/tmp/Crop_recommendation.csv')
crop_data.drop(['N', 'P', 'K', 'ph'], axis=1, inplace=True)
crop_data.head()

,temperature,humidity,rainfall,label
0,20.879744,82.002744,202.935536,rice
1,21.770462,80.319644,226.655537,rice
2,23.004459,82.320763,263.964248,rice
3,26.491096,80.158363,242.864034,rice
4,20.130175,81.604873,262.717340,rice


# Data Preprocessing

In [13]:
# Take the first eleven labels
first_11_labels = crop_data['label'].unique()[:11]

In [14]:
# Encode the labels to form 0 and 1
def encode_label(data, label):
    if data['label'] == label:
        data['label'] = 1
    else:
        data['label'] = 0
    return data

# Take 200 data from original dataset
def take_200(data, label):
    
    # Data for particular label
    specific_data = data[data['label'] == label]
    
    n = len(specific_data)
    
    # Data for beside particular label
    beside_data = data[data['label'] != label].sample(n)
    
    # Combine specific_data and beside_data
    combined_data = pd.concat([specific_data, beside_data])
    
    # Apply combined_data to convert_label function
    new_data = combined_data.apply(encode_label, label=label, axis=1)
    
    # Reset index and shuffle
    return new_data.sample(frac=1).reset_index(drop=True)

In [15]:
print('First 5 observations of data: \n')
take_200(crop_data, 'papaya').head(5)

First 5 observations of data: 



,temperature,humidity,rainfall,label
0,29.807443,91.380485,232.704613,1
1,28.282229,91.370598,179.272081,1
2,38.201682,93.973800,218.090881,1
3,42.304958,90.514318,74.876786,1
4,42.072138,92.922031,165.741297,1


## Build the Model

In [16]:
# Create model for every label-dataset
def model(data, labels):
    for label in labels:

        # Initiate data
        crop_data = take_200(data, label)
        
        # Initiate variables
        X = crop_data
        y = X.pop('label')
        
        # Split Training and Testing Dataset
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Initiate a model structure
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(64, input_shape=[3], activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        
        model.compile(optimizer=tf.keras.optimizers.Adam(), 
             loss='binary_crossentropy',
             metrics=['accuracy'])
        
        # Fitting the Model
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=0)
        
        print('{} model => Accuracy: {} & Validation Accuracy: {}\n'.format(label.capitalize(), history.history['accuracy'][-1], history.history['val_accuracy'][-1]))

In [17]:
model(crop_data, first_11_labels)

Rice model => Accuracy: 0.981249988079071 & Validation Accuracy: 1.0

Maize model => Accuracy: 0.918749988079071 & Validation Accuracy: 0.8500000238418579

Chickpea model => Accuracy: 0.981249988079071 & Validation Accuracy: 0.9750000238418579

Kidneybeans model => Accuracy: 0.96875 & Validation Accuracy: 1.0

Pigeonpeas model => Accuracy: 0.893750011920929 & Validation Accuracy: 0.875

Mothbeans model => Accuracy: 0.9375 & Validation Accuracy: 0.949999988079071

Mungbean model => Accuracy: 0.9624999761581421 & Validation Accuracy: 0.949999988079071

Blackgram model => Accuracy: 0.9937499761581421 & Validation Accuracy: 1.0

Lentil model => Accuracy: 0.8999999761581421 & Validation Accuracy: 0.875

Pomegranate model => Accuracy: 0.956250011920929 & Validation Accuracy: 0.8999999761581421

Banana model => Accuracy: 0.9312499761581421 & Validation Accuracy: 0.875



In [ ]:
# Save and convert the model
import pathlib

export_dir = '/tmp/model/'
tf.saved_model.save(model, export_dir=export_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

# Set the optimzations
# converter.optimizations = [optimization]

tflite_model = converter.convert()

tflite_model_file = pathlib.Path('/tmp/model/first-eleven-model.tflite')
tflite_model_file.write_bytes(tflite_model)